In [ ]:
%run import_libraries.ipynb

In [ ]:
%run connect_snowflake.ipynb

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow import feature_column
from tensorflow.keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import os
import h5py
from tensorflow import keras

# Import BERT Model

In [ ]:
bert_model_name = 'experts_pubmed' #'bert_multi_cased_L-12_H-768_A-12' #'small_bert/bert_en_uncased_L-4_H-512_A-8' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

# Functions

In [ ]:
class process_data:
    
    def __init__(self, topword_limit, feature_filter):
        self.topword_limit = topword_limit
        self.feature_filter = feature_filter

    def preprocess_text(self, text):
        """
        Preprocess the payer and insurance descriptions
        
        Inputs:
        
            text(str): a sentence of strings
        
        Outputs:
        
            cleaner_text(str): a cleaned sentence of strings
        
        Example:
        
            TRICARE/CHAMPUS ---> tricare champus
        
        """
        # Convert input to string
        text = str(text).lower()

        # Remove unwanted texts
        cleaner_text = re.sub(r"payer_desc_is_null", ' ', text)
        cleaner_text = re.sub(r"ins_plan_desc_is_null", ' ', cleaner_text)
        cleaner_text = re.sub(r"not provided", ' ', cleaner_text)
                
        # Remove words that start or end with numbers without punctuations
        cleaner_text = re.sub('\w*\d\w*', ' ', cleaner_text)
        
        # Remove punctuations and numbers
        cleaner_text = re.sub('[^a-zA-Z]', ' ', cleaner_text)

        # Single character removal
        cleaner_text = re.sub(r"\s+[a-zA-Z]\s+", ' ', cleaner_text)

        # Removing multiple spaces
        cleaner_text = re.sub(r'\s+', ' ', cleaner_text)

        # Remove single character
        cleaner_text = re.sub('(\b[A-Za-z] \b|\b [A-Za-z]\b)', '', cleaner_text)

        # Remove stop words
        pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
        cleaner_text = pattern.sub(' ', cleaner_text)

        return cleaner_text

    def top_words_dict(self, df):
        """"""
        top_dict = {}
        for c in df.columns:
            top = df[c].sort_values(ascending=False).head(self.topword_limit)
            top_dict[c]= list(zip(top.index, top.values))

        return top_dict

    def print_top_words(self, df):
        """"""
        for attribute_val, top_words in df.items():
            print(attribute_val)
            print(', '.join([word for word, count in top_words[0:self.topword_limit]]) + '\n')

    def freq_matrix_NA(self, df):
        """"""
        cv = CountVectorizer(stop_words='english')
        data_cv = cv.fit_transform(df[self.feature_filter])
        freq_matr = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names()).reset_index(drop=True).join(df[self.target_filter].reset_index(drop=True)).groupby([self.target_filter]).sum()
       
        return freq_matr
    
    def freq_matrix(self, df, group_by):
        """"""
        df['WORD_FREQ'] = df[self.feature_filter].str.lower().str.split(' ')
        df = df.explode('WORD_FREQ')
        df['WORD'] = df['WORD_FREQ']
        df = df.groupby([group_by,'WORD'], as_index=False)['WORD_FREQ'].count()
        df = df.loc[df['WORD'] != '', :]
        
        return df
    
    def update_text(self, df):
        """"""
        lambda_clean = lambda x: process_data.preprocess_text(self, text = x)  
        df_cleaned = df[self.feature_filter].apply(lambda_clean)
        
        return df_cleaned

        
class visualize_data:
    
    def __init__(self, initialize_data):
        super().__init__(initialize_data.file_date, initialize_data.file_path, initialize_data.file_tab, initialize_data.file_values, initialize_data.target_col_name, initialize_data.features)
    
    def plot_data(self, df_labels):
        """"""
        fig_size = plt.rcParams["figure.figsize"]
        fig_size[0] = 6
        fig_size[1] = 5
        plt.rcParams["figure.figsize"] = fig_size

        df_labels.sum(axis=0).plot.bar()
        
        

In [ ]:
def bert_embeddings(sentences):
    """
    Return BERT embeddings for the input description
    
    Inputs:
    
        sentences(list): sentences of strings
    
    Outputs:
    
        tf.tensor: Bert embeddings
    
    Example:
    
        >> bert_embeddings(['hello', 'world'])

        <tf.Tensor: shape=(2, 512), dtype=float32, numpy=
        array([[ 0.3973088 ,  0.92228746,  0.18193923, ...,  0.40281808, -0.51239616, -0.67238265],
               [ 0.9519767 ,  0.6982696 , -0.09281926, ..., -0.38502616, -0.28820902, -0.79655826]], dtype=float32)>
    
    """
    
    bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
    bert_model = hub.KerasLayer(tfhub_handle_encoder)

    preprocessed_text = bert_preprocess_model(sentences)
    
    return bert_model(preprocessed_text)['pooled_output']

In [ ]:
def nnet_pipeline(drop_out):
    """
    Execute the neuralnet pipeline. Preprocess the inputs, encode the preprocessed inputs, and feed into neural net layers
    
    Inputs:
    
        drop_out(float): A probablity value representating the percentage of random sample to drop out
    
    Outputs:
    
        Model
    
    """
    
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessor = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessor(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    nnet = outputs['pooled_output']
    nnet = tf.keras.layers.Dropout(drop_out, name="dropout")(nnet)
    nnet = tf.keras.layers.Dense(len(labels), activation='softmax', name="output")(nnet)
    model = tf.keras.Model(text_input, nnet)
    
    return model

In [ ]:
def execute_model(x_train, y_train, x_test, y_test, n_epochs = 3, optimizer = "adam", loss_function = "categorical_crossentropy", patience = 3, drop_out = 0.1):
    """
    Execute the model and return the trained model
    
    Inputs:
    
        x_train(tf): tensorflow train features
        y_train(tf): tensorflow train labels
        x_test(tf): tensorflow test features
        y_test(tf): tensorflow train labels
        n_epochs(int): the number of train data points over the batch size
        optimizer(str): optimization algorithm
        loss_function(str): evaluation metric for the predictions
        patience(int): early exit for inference to improve efficiency
        drop_out(float): A probablity value representating the percentage of random sample to drop out
        
    Outputs:
    
        model_fit: trained model
    
    """
    
    model = nnet_pipeline(drop_out)
        
    earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", 
                                                          patience = patience,
                                                          restore_best_weights = True)
    model.compile(optimizer = optimizer,
                  loss = loss_function,
                  metrics = [tf.keras.metrics.CategoricalAccuracy(name="accuracy")])
        
    model_fit = model.fit(x_train, 
                          y_train, 
                          epochs = n_epochs,
                          validation_data = (x_test, y_test),
                          callbacks = [earlystop_callback])
    
    return model_fit, model

In [ ]:
def get_prediction(description):
    """
    Return the predictions
    
    Inputs:
    
        description(list): sentences of strings
        
    Outputs:
    
        predictions(list)
    
    """
    
    return [np.argmax(pred) for pred in model.predict(description)]


# Import Data

In [ ]:
my_data = import_my_data(limit = 10000000)

In [ ]:
np.shape(my_data)

# Preprocess the data

In [ ]:
# Initialize
topword_limit = 50
feature_filter = 'description'

In [ ]:
text_processor = process_data(topword_limit=topword_limit, feature_filter=feature_filter)

In [ ]:
# process and clean the insurance and payer descriptions
my_data_cleaned = pd.DataFrame(text_processor.update_text(df = my_data))

In [ ]:
labels = list(my_data['labels'].unique())
labels.sort(reverse=False)

In [ ]:
labels_dict = {'Commercial': 0,
                 'Medicaid': 1,
                 'Medicare Advantage': 2,
                 'Medicare Traditional': 3,
                 'Self-pay': 4,
                 'VA': 5}

In [ ]:
my_data_cleaned['labels'] = my_data['labels'].map(labels_dict) # encode the labels

# Split into train and test set

In [ ]:
y = tf.keras.utils.to_categorical(my_data_cleaned["labels"].values, num_classes=len(labels))

x_train, x_test, y_train, y_test = train_test_split(my_data_cleaned['description'], y, test_size=0.20)

# Train

In [ ]:
model_fit, model = execute_model(x_train, y_train, x_test, y_test, n_epochs = 2, optimizer = "adam", loss_function = "categorical_crossentropy", patience = 3, drop_out = 0.3)

# Predict

In [ ]:
pred_dict = {v: k for k, v in labels_dict.items()} # flip the labels dictionary

y_test_actual = []

for i in range(np.shape(y_test)[0]):
    y_test_actual.append(list(y_test[i]).index(1))

check_pred = pd.DataFrame()
check_pred['description'] = list(x_test)
check_pred['prediction'] = get_prediction(list(x_test))
check_pred['prediction_label'] = check_pred['prediction'].map(pred_dict)
check_pred['actual_value'] = y_test_actual
check_pred['actual_label'] = check_pred['actual_value'].map(pred_dict)
check_pred['prediction_correct'] = check_pred['prediction_label'] == check_pred['actual_label']

In [ ]:
print("Accuracy: " + str(sum(check_pred['prediction_correct']) / np.shape(check_pred)[0]))

# Predict on out-of-sample data

In [ ]:
test_set = import_my_data(limit=1000000, data_type='test')['description']

In [ ]:
np.shape(test_set)

In [ ]:
check_pred_test = pd.DataFrame()
check_pred_test['description'] = list(test_set)
check_pred_test['prediction'] = get_prediction(list(test_set))
check_pred_test['prediction_label'] = check_pred_test['prediction'].map(pred_dict)

In [ ]:
check_pred_test.to_excel('C:/Users/' + my_username + '/Downloads/' + 'my_test_set' + '.xlsx')

# Save Predictions

In [ ]:
my_username = 'user_name'
save_filename = 'my_test_data_pred'

In [ ]:
user_input1 = input("Save predictions? [y/n]: ")

if user_input1 == 'y':
    check_pred.to_excel('C:/Users/' + my_username + '/Downloads/' + save_filename + '.xlsx')

# Save and Load Model

In [ ]:
save_modelname = 'ky_my_model_v1'
import_modelname = 'ky_my_model_v1'

In [ ]:
user_input2 = input("Save model? [y/n]: ")

if user_input2 == 'y':
    model.save('C:/Users/'+ my_username + '/Downloads/saved_model/' + save_modelname + '.hdf5')

In [ ]:
user_input3 = input("Load model? [y/n]: ")

if user_input3 == 'y':
    import_model_path = 'c:/users/' + my_username + '/Downloads/saved_model/' + import_modelname + '.hdf5'
    imported_model = load_model(import_model_path, custom_objects={'KerasLayer':hub.KerasLayer})